In [2]:
%pip install scikit-learn
# %pip install scipy

Defaulting to user installation because normal site-packages is not writeable
You should consider upgrading via the '/Applications/Xcode.app/Contents/Developer/usr/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [3]:
import numpy as np
import matplotlib.pyplot as plt
import random
from sklearn.datasets import make_blobs
import statistics
from scipy.spatial import Voronoi, voronoi_plot_2d

In [4]:
candidates = 4
voters = 10000

#cost for candidate x is sum of all distances from x to a voter on plot
#optimum candidate is candidate x s.t. opt_cand = argmin(Cost(x in X))
#Distortion is worst case ratio by candidate chosen by voter profiles over optimum candidate costs


In [5]:
def calculate_cost(v_x, v_y, c_x, c_y):
    # Calculate cost for each candidate
    costs = []
    for x2, y2 in zip(c_x, c_y):
        sum = 0
        for x, y in zip(v_x, v_y):
            distance = dist([x,y],[x2,y2])
            sum += distance
        costs.append(sum)
        print(f'Cost of candidate = {sum}')
    # Return minimum cost
    return costs


In [6]:
import math
from math import dist
#sklearn make blobs

def gen_and_plot_points(voters, candidates):
    x1 = []
    y1 = []
    for _ in range(voters):
        x1.append(random.randint(-60, 60))
        y1.append(random.randint(-60, 60))

    x2 = []
    y2 = []
    for _ in range(candidates):
        x2.append(random.randint(-50, 50))
        y2.append(random.randint(-50, 50))

    for i in range(candidates):
      print(f'canidate {i} coordinate: ({x2[i]}, {y2[i]})')

    # plt.scatter(x1, y1, color='blue', label='Voters')
    # plt.scatter(x2, y2, color='red', label='Candidates')
    # plt.title('Randomly Generated Points')
    # plt.legend()  # Show legend
    # plt.grid(True)
    # plt.show()
    return x1, y1, x2, y2



v_x, v_y, c_x, c_y = gen_and_plot_points(voters, candidates)



# X1, Y1, Centers = make_blobs(n_samples=voters, n_features=2, centers=candidates, center_box=(-20.0,20.0), cluster_std=3.8, return_centers=True)
# plt.scatter(X1[:,0], X1[:,1], marker='o', c=Y1, s=25, edgecolor='k')


#For every ranking color a point a color for a different plot. Color every voter by their ranking (dont need to stick with the blobs)
#assign each ranking a number. number corresponds to color those will be new labels instead of Y1

canidate 0 coordinate: (34, 46)
canidate 1 coordinate: (24, 41)
canidate 2 coordinate: (-27, -40)
canidate 3 coordinate: (48, -37)


In [7]:
# print(X1)
# print(f'X1 shape is: {X1.shape}')
#Cost(Candidate 0) = Sum of distances from vi to candidate c0

# plt.show()

def get_preferences(v_x, v_y, c_x, c_y):
  preferences = []
  for x, y in zip(v_x, v_y):
      # Create a list of tuples [(distance, original_index), ...]
      distance_and_index = []
      for index, (x2, y2) in enumerate(zip(c_x, c_y)):
          distance = dist([x, y], [x2, y2])
          distance_and_index.append((distance, index))

      # Sort based on distances and extract the ranking by their original indices
      distance_and_index.sort()  # Sorts based on distance because it's the first element of the tuple
      ranked_preferences = [index for _, index in distance_and_index]
      preferences.append(ranked_preferences)
  return preferences

preferences = get_preferences(v_x, v_y, c_x, c_y)

# preferences

In [8]:
costs = calculate_cost(v_x, v_y, c_x, c_y)


costs_array = np.array(costs)
min_index = np.argmin(costs_array)
# rank = [sorted(costs).index(x) for x in costs]
# index = np.where(rank == 0)
print(min_index)
#Optimal candidate is first index
print(f'Optimal Candidate Position: [{c_x[min_index]}, {c_y[min_index]}]')

Cost of candidate = 687902.1273109071
Cost of candidate = 621830.9432816535
Cost of candidate = 627336.7314089786
Cost of candidate = 709026.1316324715
1
Optimal Candidate Position: [24, 41]


In [33]:
preferences[1593]

[3, 1, 2, 0]

In [9]:
#Debug code
from matplotlib.patches import Patch
import itertools

def plot_voronoi(x1, y1, x2, y2, preferences):
    # Assign colors to each candidate
    colors = [
        'red', 'green', 'blue', 'yellow', 'orange', 'purple', 'cyan', 'magenta',
        'lime', 'pink', 'teal', 'lavender', 'brown', 'beige', 'maroon', 'green',
        'coral', 'olive', 'navy', 'grey', 'black', 'white', 'gold', 'silver'
    ]
    permutations = list(itertools.permutations([0, 1, 2, 3]))

    # Ensure there are enough colors
    assert len(colors) >= len(permutations), "Not enough colors for all permutations"

    # Create the dictionary mapping permutations to colors
    perm_to_color = {perm: colors[i] for i, perm in enumerate(permutations)}

    # Example usage: Print the mapping
    # for perm, color in perm_to_color.items():
    #     print(f"Permutation {perm} is mapped to color {color}")

    # Add more colors if needed
    # Plot the points
    for i in range(len(x1)):
        # print(i)
        voter_color = perm_to_color[tuple(preferences[i])]  # Color based on first preference
        plt.scatter(x1[i], y1[i], color=voter_color)
    for i in range(len(x2)):
        plt.scatter(x2[i], y2[i], color='brown', label=f'Candidate {i+1}')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.title('Voronoi Voter Plot')
    # plt.legend()  # Show legend
    plt.grid(True)
    plt.show()

In [21]:
plot_voronoi(v_x, v_y, c_x, c_y)

str

In [11]:
num_candidates = len(preferences[0])

# Random Dictator Rule
def random_dictator(preferences):
    dictator = random.choice(preferences)
    return dictator[0]

# Plurality Voting Rule
def plurality(preferences, num_candidates):
    # Initialize vote counts
    vote_counts = [0] * num_candidates
    for ranking in preferences:
        vote_counts[ranking[0]] += 1
    # Candidate with the most votes wins
    winner = vote_counts.index(max(vote_counts))
    return winner

# Borda Count Voting Rule
def borda(preferences, num_candidates):
    # Initialize Borda scores
    scores = [0] * num_candidates
    for ranking in preferences:
        for index, candidate in enumerate(ranking):
            scores[candidate] += num_candidates - index - 1
    # Candidate with the highest Borda score wins
    winner = scores.index(max(scores))
    return winner

# Determine the optimal candidate based on each rule
random_dictator_winner = random_dictator(preferences)
plurality_winner = plurality(preferences, num_candidates)
borda_winner = borda(preferences, num_candidates)

print(f'Random Dict: {random_dictator_winner}')
print(f'Plurality: {plurality_winner}')
print(f'Borda: {borda_winner}')

Random Dict: 0
Plurality: 2
Borda: 1


In [12]:
epochs = 50
fig, axes = plt.subplots(1, epochs, figsize=(5 * epochs, 5))
cost_ratios = []
for j in range(epochs):
    v_x, v_y, c_x, c_y = gen_and_plot_points(voters, candidates)
    preferences = get_preferences(v_x, v_y, c_x, c_y)
    costs = calculate_cost(v_x, v_y, c_x, c_y)
    costs_array = np.array(costs)
    min_index = np.argmin(costs_array)
    print(min_index)
    opt_can = min_index
    #Get voting rule winners
    random_dictator_winner = random_dictator(preferences)
    plurality_winner = plurality(preferences, num_candidates)
    borda_winner = borda(preferences, num_candidates)
    #Optimal candidate is min index
    print(f'Optimal Candidate Position: [{c_x[min_index]}, {c_y[min_index]}]')
    # Assign colors to each candidate
    plurality_cr = float(costs[plurality_winner])/float(costs[opt_can])
    borda_cr = float(costs[borda_winner])/float(costs[opt_can])
    randomd_cr = float(costs[random_dictator_winner])/float(costs[opt_can])
    cost_ratios.append([plurality_cr, borda_cr, randomd_cr])
    print(f'Cost Ratios: Plurality - {plurality_cr}, Borda - {borda_cr}, Random dictator - {randomd_cr}')

    colors = [
        'red', 'green', 'blue', 'yellow', 'orange', 'purple', 'cyan', 'magenta',
        'lime', 'pink', 'teal', 'lavender', 'brown', 'beige', 'maroon', 'green',
        'coral', 'olive', 'navy', 'grey', 'black', 'white', 'gold', 'silver'
    ]
    permutations = list(itertools.permutations([0, 1, 2, 3]))

    # Ensure there are enough colors
    assert len(colors) >= len(permutations), "Not enough colors for all permutations"

    # Create the dictionary mapping permutations to colors
    perm_to_color = {perm: colors[i] for i, perm in enumerate(permutations)}

    # Example usage: Print the mapping
    # for perm, color in perm_to_color.items():
    #     print(f"Permutation {perm} is mapped to color {color}")

    # Add more colors if needed
    # Plot the points
    for i in range(len(v_x)):
        # print(i)
        voter_color = perm_to_color[tuple(preferences[i])]  # Color based on first preference
        axes[j].scatter(v_x[i], v_y[i], color=voter_color)
    for i in range(len(c_x)):
        axes[j].scatter(c_x[i], c_y[i], color='brown', label=f'Candidate {i+1}')
    axes[j].set_xlabel('X')
    axes[j].set_ylabel('Y')
    axes[j].set_title(f"Voronoi Plot {i + 1}")
    # plt.legend()  # Show legend
    plt.grid(True)

plt.tight_layout()
plt.show()



canidate 0 coordinate: (-4, -49)
canidate 1 coordinate: (49, -26)
canidate 2 coordinate: (-22, 50)
canidate 3 coordinate: (8, 7)
Cost of candidate = 634680.851434445
Cost of candidate = 675744.4477645331
Cost of candidate = 665531.1934181346
Cost of candidate = 468391.27789388
3
Optimal Candidate Position: [8, 7]
Cost Ratios: Plurality - 1.0, Borda - 1.0, Random dictator - 1.3550227798610717
canidate 0 coordinate: (19, 50)
canidate 1 coordinate: (-26, -5)
canidate 2 coordinate: (-15, -6)
canidate 3 coordinate: (-7, 2)
Cost of candidate = 665327.3258206709
Cost of candidate = 509494.7136332897
Cost of candidate = 477588.2657586137
Cost of candidate = 463375.7829580187
3
Optimal Candidate Position: [-7, 2]
Cost Ratios: Plurality - 1.0995281418913714, Borda - 1.0, Random dictator - 1.4358267097461777
canidate 0 coordinate: (-47, -43)
canidate 1 coordinate: (-13, -5)
canidate 2 coordinate: (-17, 49)
canidate 3 coordinate: (20, -45)
Cost of candidate = 739902.1774255529
Cost of candidate = 

In [1]:
plt.savefig("100-trial-voting-rules.png",format='png')

NameError: name 'plt' is not defined

In [ ]:
trials = list(range(1, epochs + 1))

plurality_list = []
borda_list = []
rand_dict_list = []

for ratios in cost_ratios:
    plurality_list.append(ratios[0])
for ratios in cost_ratios:
    borda_list.append(ratios[1])
for ratios in cost_ratios:
    rand_dict_list.append(ratios[2])

for i in range(1,epochs+1):
    plt.plot(trials, plurality_list, label="Plurality" if i == 1 else "", marker='o')
    plt.plot(trials, borda_list, label="Borda" if i == 1 else "", marker='o')
    plt.plot(trials, rand_dict_list, label="Random Dictator" if i == 1 else "", marker='o')

# Label the plot
plt.xlabel('Trial')
plt.ylabel('Cost Ratio')
plt.title('Cost Ratios of Different Voting Rules Over Trials')
plt.legend()
plt.grid(True)

# Show the plot
plt.show()
plt.savefig("Cost-Ratio-LP.png")

In [ ]:
#Use Vote-kit to score things
#Check slack for link

for j in range(len(preferences[0])):
    ranks = []
    s0 = s1 = s2 = s3 = s4 = 0
    for i in range(len(preferences)):
        if (preferences[i][j] == 0):
            s0 += 1
        elif (preferences[i][j] == 1):
            s1 += 1
        elif (preferences[i][j] == 2):
            s2 += 1
        elif (preferences[i][j] == 3):
            s3 += 1
        elif (preferences[i][j] == 4):
            s4 += 1
    print(f"rank {j} has {s0} 0's, {s1} 1's, {s2} 2's, {s3} 3's")

rank 0 has 3687 0's, 1104 1's, 522 2's, 4687 3's
rank 1 has 2494 0's, 4763 1's, 2743 2's, 0 3's
rank 2 has 2205 0's, 1271 1's, 2356 2's, 4168 3's
rank 3 has 1614 0's, 2864 1's, 4379 2's, 1143 3's


In [ ]:
c_1 = 0
c_2 = 0
c_3 = 0
c_4 = 0
c_5 = 0
optprofile = []
for preference in zip(preferences):
    # print(preference[0][2])
    profilez = []
    for i in range(candidates):
        profilez.append(preference[0][i])
    optprofile.append(profilez)
    if preference[0][0] == 0:
        c_1 += 1
    elif preference[0][0] == 1:
        c_2 += 1
    elif preference[0][0] == 2:
        c_3 += 1
    elif preference[0][0] == 3:
        c_4 += 1
    elif preference[0][0] == 4:
        c_5 += 1

# print(optprofile[4003])""

final = []
for i in range(candidates):
    for j in range(voters):
        final.append(optprofile[j][i])
    vp = statistics.mode(optprofile[i])
    final.append(vp)


print(f'cand 1 has fc: {(float(c_1)/voters)*100}% and {c_1} votes')
print(f'cand 2 has fc: {(float(c_2)/voters)*100}% and {c_2} votes')
print(f'cand 3 has fc: {(float(c_3)/voters*100)}% and {c_3} votes')
print(f'cand 4 has fc: {(float(c_4)/voters*100)}% and {c_4} votes')
print(f'cand 5 has fc: {(float(c_5)/voters*100)}% and {c_5} votes')
print('\n')
# print(f'voting profile is {final}')

cand 1 has fc: 11.450000000000001% and 1145 votes
cand 2 has fc: 28.71% and 2871 votes
cand 3 has fc: 40.589999999999996% and 4059 votes
cand 4 has fc: 19.25% and 1925 votes
cand 5 has fc: 0.0% and 0 votes


